In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/titanic/train.csv
/kaggle/input/titanic/test.csv
/kaggle/input/titanic/gender_submission.csv


Uploading the train dataset to a csv file

In [2]:
train_loc="/kaggle/input/titanic/train.csv"
train_data=pd.read_csv(train_loc)
train_data.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


importing the test data

In [3]:
test_data=pd.read_csv('/kaggle/input/titanic/test.csv')

using minmax scaling on fare


In [4]:
from sklearn.preprocessing import MinMaxScaler
scaler=MinMaxScaler()
X1=train_data.loc[:,['Age','Fare']]
X1.head()

,Age,Fare
0,22.0,7.2500
1,38.0,71.2833
2,26.0,7.9250
3,35.0,53.1000
4,35.0,8.0500


fitting the Scaler

In [5]:
scaler.fit(X1)
X1_scaled=scaler.transform(X1)
X1_scaled=pd.DataFrame(X1_scaled,columns=X1.columns)
X1_scaled.head()

,Age,Fare
0,0.271174,0.014151
1,0.472229,0.139136
2,0.321438,0.015469
3,0.434531,0.103644
4,0.434531,0.015713


categorical encoding 

In [6]:
y=train_data["Survived"]
features = ["Pclass", "Sex", "SibSp", "Parch"]
X2 = pd.get_dummies(train_data[features])
X2_test = pd.get_dummies(test_data[features])
X2.head()

,Pclass,SibSp,Parch,Sex_female,Sex_male
0,3,1,0,0,1
1,1,1,0,1,0
2,3,0,0,1,0
3,1,1,0,1,0
4,3,0,0,0,1


Scaling down of the float features of the test dataset

In [7]:
X1_test=test_data.loc[:,['Age','Fare']]
X1_test.head()

,Age,Fare
0,34.5,7.8292
1,47.0,7.0000
2,62.0,9.6875
3,27.0,8.6625
4,22.0,12.2875


fitting the scaler accordingly

In [8]:
scaler.fit(X1_test)
X1_test_scaled=scaler.transform(X1_test)
X1_test_scaled=pd.DataFrame(X1_test_scaled,columns=X1_test.columns)
X1_test_scaled.head()

,Age,Fare
0,0.452723,0.015282
1,0.617566,0.013663
2,0.815377,0.018909
3,0.353818,0.016908
4,0.287881,0.023984


joining the train and test datasets

In [9]:
X=pd.concat([X1_scaled,X2],axis=1)
X_test=pd.concat([X1_test_scaled,X2_test],axis=1)

The final training dataset displayed

In [10]:
X.head()

,Age,Fare,Pclass,SibSp,Parch,Sex_female,Sex_male
0,0.271174,0.014151,3,1,0,0,1
1,0.472229,0.139136,1,1,0,1,0
2,0.321438,0.015469,3,0,0,1,0
3,0.434531,0.103644,1,1,0,1,0
4,0.434531,0.015713,3,0,0,0,1


the final Test dataset displayed

In [11]:
X_test.head()

,Age,Fare,Pclass,SibSp,Parch,Sex_female,Sex_male
0,0.452723,0.015282,3,0,0,0,1
1,0.617566,0.013663,3,1,0,1,0
2,0.815377,0.018909,2,0,0,0,1
3,0.353818,0.016908,3,0,0,0,1
4,0.287881,0.023984,3,1,1,1,0


In [12]:
y.head()

0    0
1    1
2    1
3    1
4    0
Name: Survived, dtype: int64

dealing with missing values

In [13]:
X.isnull().sum()

Age           177
Fare            0
Pclass          0
SibSp           0
Parch           0
Sex_female      0
Sex_male        0
dtype: int64

In [14]:
X=X.drop(columns=['Age'])
X.head()

,Fare,Pclass,SibSp,Parch,Sex_female,Sex_male
0,0.014151,3,1,0,0,1
1,0.139136,1,1,0,1,0
2,0.015469,3,0,0,1,0
3,0.103644,1,1,0,1,0
4,0.015713,3,0,0,0,1


In [15]:
X_test.isnull().sum()

Age           86
Fare           1
Pclass         0
SibSp          0
Parch          0
Sex_female     0
Sex_male       0
dtype: int64

Data Cleaning 

In [16]:
X_test=X_test.drop(columns=['Age'])
X_test.head()

,Fare,Pclass,SibSp,Parch,Sex_female,Sex_male
0,0.015282,3,0,0,0,1
1,0.013663,3,1,0,1,0
2,0.018909,2,0,0,0,1
3,0.016908,3,0,0,0,1
4,0.023984,3,1,1,1,0


checking for any remaining null values

checking the dataset now

In [17]:
X_test.head()

,Fare,Pclass,SibSp,Parch,Sex_female,Sex_male
0,0.015282,3,0,0,0,1
1,0.013663,3,1,0,1,0
2,0.018909,2,0,0,0,1
3,0.016908,3,0,0,0,1
4,0.023984,3,1,1,1,0


using simple column filling

In [18]:
X_test=X_test.fillna(method='bfill', axis=0).fillna(0)


In [19]:
X_test.isnull().sum()

Fare          0
Pclass        0
SibSp         0
Parch         0
Sex_female    0
Sex_male      0
dtype: int64

In [20]:
from xgboost import XGBClassifier
model=XGBClassifier(n_estimators=500)


In [21]:
model.fit(X,y)

XGBClassifier(base_score=0.5, booster='gbtree', callbacks=None,
              colsample_bylevel=1, colsample_bynode=1, colsample_bytree=1,
              early_stopping_rounds=None, enable_categorical=False,
              eval_metric=None, gamma=0, gpu_id=-1, grow_policy='depthwise',
              importance_type=None, interaction_constraints='',
              learning_rate=0.300000012, max_bin=256, max_cat_to_onehot=4,
              max_delta_step=0, max_depth=6, max_leaves=0, min_child_weight=1,
              missing=nan, monotone_constraints='()', n_estimators=500,
              n_jobs=0, num_parallel_tree=1, predictor='auto', random_state=0,
              reg_alpha=0, reg_lambda=1, ...)

making predictions

In [22]:
predictions=model.predict(X_test)

Uploading the csv with the necessary predictions

In [23]:
output = pd.DataFrame({'PassengerId': test_data.PassengerId, 'Survived': predictions})
output.to_csv('submission.csv', index=False)
print("Your submission was successfully saved!")

Your submission was successfully saved!
